<a href="https://colab.research.google.com/github/Siu0901/Plain_Project_recommendation_System/blob/main/%ED%94%8C%EB%A0%88%EC%9D%B8_%EC%B6%94%EC%B2%9C_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

In [123]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
# model = SentenceTransformer("jhgan/ko-sbert-sts") # 옌 한국어임
model = SentenceTransformer("all-MiniLM-L6-v2")

In [125]:
data = pd.read_csv("tmdb_5000_movies.csv")
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [126]:
data['overview'] = data['overview'].fillna('')

In [127]:
genre = data['genres']
genre.head(2)

,genres
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."


In [128]:
# 지옥의 장르 뽑아내기
genre_list = []
replace_list = ['[','{','"',':',']','}',',','1','2','3','4','5','6','7','8','9','0']
alp_list = ['id','name']

for i in genre:
  g = i
  for j in replace_list:
    g = g.replace(j,'')
  genre_list.append(g)

re_ge = []
for i in range(len(genre_list)):
  re_ge.append([item for item in genre_list[i].replace('name','').replace('id','').split(' ') if item])

re_ge[0]

['Action', 'Adventure', 'Fantasy', 'Science', 'Fiction']

In [140]:
# 자카드 유사도 구하기
def jac(list1, list2):
  s1 = set(list1)
  s2 = set(list2)
  union_size = len(s1.union(s2))
  if union_size == 0:
    return 0.0
  return float(len(s1.intersection(s2)) / union_size)

jac_list = []
for i in range(len(re_ge)):
  jac_list.append([])
  for j in range(len(re_ge)):
    jac_list[i].append(jac(re_ge[i],re_ge[j]))

jac_list = np.array(jac_list)

In [144]:
threshold = 0.2
candidates_dict = {}

for i in range(len(jac_list)):
    candidates = [
        j for j, sim in enumerate(jac_list[i])
        if sim >= threshold and j != i
    ]
    candidates_dict[i] = candidates

print(candidates_dict[0])  # 0번 책의 후보군

[1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 59, 61, 62, 63, 64, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 121, 122, 123, 125, 126, 127, 129, 131, 132, 134, 138, 139, 143, 147, 148, 149, 150, 151, 152, 153, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 182, 183, 184, 185, 186, 187, 191, 193, 195, 196, 197, 198, 199, 200, 203, 205, 206, 207, 208, 210, 212, 213, 215, 216, 217, 219, 220, 222, 223, 224, 225, 228, 229, 230, 232, 233, 234, 235, 237, 238, 239, 240, 241, 242, 243, 245, 249, 254, 256, 257, 259, 260, 262, 263, 266, 267, 270, 271, 274, 275, 276, 277, 278, 279, 289, 290, 291, 292, 294, 296, 299, 300, 301, 302, 304, 305, 306, 307, 309, 310, 311, 312, 31

In [163]:
# 일단 대충 테스트
n = input('영화 제목 : ') # Avatar
a = data[data['title'] == n].index[0]
print(candidates_dict[a])
print(data['overview'][candidates_dict[a][0]])
sen = [data['overview'][candidates_dict[a][0]],data['overview'][candidates_dict[a][1]],data['overview'][candidates_dict[a][2]]]
embeddings = model.encode(sen)
print(embeddings.shape)
similarities = model.similarity(embeddings, embeddings) # 여기서 이미 코사인 유사도 해둠
print(np.array(similarities))

영화 제목 : Avatar
[1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 59, 61, 62, 63, 64, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 121, 122, 123, 125, 126, 127, 129, 131, 132, 134, 138, 139, 143, 147, 148, 149, 150, 151, 152, 153, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 182, 183, 184, 185, 186, 187, 191, 193, 195, 196, 197, 198, 199, 200, 203, 205, 206, 207, 208, 210, 212, 213, 215, 216, 217, 219, 220, 222, 223, 224, 225, 228, 229, 230, 232, 233, 234, 235, 237, 238, 239, 240, 241, 242, 243, 245, 249, 254, 256, 257, 259, 260, 262, 263, 266, 267, 270, 271, 274, 275, 276, 277, 278, 279, 289, 290, 291, 292, 294, 296, 299, 300, 301, 302, 304, 305, 306, 307, 309, 31